In [1]:



import transformers


In [2]:
import torch
from typing import Any
from dataclasses import dataclass

LLMModel = Any


# TODO: Cite https://github.com/abazarova/tda4hallucinations/
@dataclass
class TokenwiseEntropy:
    llm_model: LLMModel
    device: str = "cuda"

    @torch.no_grad()
    def calculate(self, input_ids) -> float:
        token_distribution = self._get_token_distribution(input_ids)
        entropy = self._compute_entropy_from_logits(token_distribution)
        return entropy.detach().cpu().item()

    def _get_token_distribution(self, input_ids) -> torch.Tensor:
        # Yield the output of the model for the current example
        output = self.llm_model(
            input_ids,
            output_hidden_states=True,
            output_attentions=False,
        )

        return output.logits[0, -1:]

    def _compute_entropy_from_logits(self, logits: torch.Tensor) -> torch.Tensor:
        """
        Compute entropy from logits.

        Parameters:
        ----------
        logits : torch.Tensor
            Logits from the model.

        Returns:
        -------
        torch.Tensor
            Entropy values.
        """
        probabilities = torch.softmax(logits, dim=-1)
        log_probabilities = torch.log(probabilities + 1e-12)
        entropies = -torch.sum(probabilities * log_probabilities, dim=-1)
        return entropies[0]

In [3]:
from typing import List


def get_sys_prompt(subject: str | None = None):
    if subject is not None:
        sys_msg = f"The following are multiple choice questions about {subject}."
    else:
        sys_msg = "The following are multiple choice questions."

    sys_msg += " Write only the answer number and nothing else."
    return sys_msg


option_ids = [str(i + 1) for i in range(20)]


def get_user_prompt(question: str, options: List[str]):
    options_str = "\n".join([f"{option_id}. {answer}".strip() for option_id, answer in zip(option_ids, options)])
    user_prompt = f"Question: {question.strip()}\nOptions:\n{options_str}\nChoose one of the answers. Write down ONLY the NUMBER of the correct answer and nothing else."
    return user_prompt


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tokenizers import Tokenizer
# from tokenizers.models import BPE
# from tokenizers.pre_tokenizers import Whitespace

MODEL_NAME = "Mistral-Small-24B-Instruct-2501"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = Whitespace()


model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", trust_remote_code=True,) 

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
print(model.get_memory_footprint() / 10**9)
print(next(model.parameters()).device)

94.28962304
cuda:0


In [6]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")

In [7]:
!nvidia-smi

Thu Feb  6 22:13:44 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:0B:00.0 Off |                    0 |
| N/A   33C    P0              66W / 400W |  22069MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [9]:
import pandas as pd
import ast
import csv
from tqdm import tqdm
import gc
import os, sys


DUMP_EVERY = 100
invalid_answers = 0


def estimate_dataset(
    df, model, tokenizer, get_subject_from_row, get_question_from_row, get_options_from_row, verify_answer, out_filename
):
    global invalid_answers

    model_name = model.config_class().model_type
    print(model_name)

    field_ans_correct = f"entropy_ans_correct_{model_name}"
    field_ans_output = f"entropy_ans_output_{model_name}"
    field_entropy_value = f"field_entropy_value_{model_name}"

    if field_ans_correct not in df.columns:
        df[field_ans_correct] = False
    if field_ans_output not in df.columns:
        df[field_ans_output] = ""
    if field_entropy_value not in df.columns:
        df[field_entropy_value] = 0.0

    entropy_estimator = TokenwiseEntropy(llm_model=model, device=DEVICE)

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if df.at[index, field_ans_output] != "" and df.at[index, field_ans_output] != "INVALID":
            continue

        print(f"loop {index} -> start: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")

        sys_prompt = get_sys_prompt(get_subject_from_row(row))
        user_prompt = get_user_prompt(get_question_from_row(row), get_options_from_row(row))
        messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt},
        ]
        formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = tokenizer(formatted_prompt, return_tensors="pt").to(DEVICE)

        outputs = model.generate(**inputs, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)
        print(f"loop {index} -> after generate: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")

        input_length = inputs.input_ids.shape[1]
        answer_raw = outputs[0, input_length:]
        answer = tokenizer.decode(answer_raw, skip_special_tokens=True)
        if answer in option_ids:
            entropy = entropy_estimator.calculate(outputs)
            print(f"loop {index} -> after entropy: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")
            df.at[index, field_entropy_value] = entropy
            df.at[index, field_ans_output] = answer
            df.at[index, field_ans_correct] = verify_answer(row, answer)
        else:
            df.at[index, "entropy_ans_output"] = "INVALID"
            invalid_answers += 1

        print(f"Answer: {answer}\nEntropy: {entropy}\nis_correct: {df.at[index, field_ans_correct]}\n\n")

        if index % DUMP_EVERY == 0:
            df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)

        gc.collect()
        torch.cuda.empty_cache()

    df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)
    print(f"Processed dataset {out_filename}. Total entries: {df.shape[0]}. Invalid answers: {invalid_answers}")
    return df


ORIGINAL_DATASET = "./mmlu_pro_stem"
original_filename = f"{ORIGINAL_DATASET}.tsv"
out_filename = f"{ORIGINAL_DATASET}_w_mistral_entropy.tsv"

if os.path.isfile(out_filename):
    df = pd.read_csv(
        out_filename,
        sep="\t",
        header=0,
        quoting=csv.QUOTE_NONE,
        quotechar="",
        escapechar="\\",
    )
else:
    df = pd.read_csv(
        original_filename,
        sep="\t",
        header=0,
    ).head(6016)
# df = df.head(10)


def verify_model_answer(row, model_answer):
    try:
        return int(row["answer_index"]) + 1 == int(model_answer)
    except:
        return False


estimate_dataset(
    df=df,
    model=model,
    tokenizer=tokenizer,
    get_subject_from_row=lambda row: row["base_cluster"],
    get_question_from_row=lambda row: row["question"],
    get_options_from_row=lambda row: ast.literal_eval(row["options"]),
    verify_answer=verify_model_answer,
    out_filename=out_filename,
)

mistral


  0%|          | 0/6016 [00:00<?, ?it/s]

loop 0 -> start: 94.28962304 GB
loop 0 -> after generate: 94.28962304 GB
loop 0 -> after entropy: 94.28962304 GB
Answer: 3
Entropy: 0.3901011347770691
is_correct: True




  0%|          | 1/6016 [00:03<5:46:45,  3.46s/it]

loop 1 -> start: 94.28962304 GB
loop 1 -> after generate: 94.28962304 GB


...

100%|█████████▉| 6015/6016 [3:41:43<00:03,  3.15s/it]

loop 6014 -> after entropy: 94.28962304 GB
Answer: 4
Entropy: 0.33550432324409485
is_correct: False


loop 6015 -> start: 94.28962304 GB
loop 6015 -> after generate: 94.28962304 GB


100%|██████████| 6016/6016 [3:41:47<00:00,  2.21s/it]

loop 6015 -> after entropy: 94.28962304 GB
Answer: 2
Entropy: 0.2471989542245865
is_correct: False


Processed dataset ./mmlu_pro_stem_w_mistral_entropy.tsv. Total entries: 6016. Invalid answers: 299


,src,answer,options,category,question,cot_content,question_id,answer_index,total_tokens,meta_cluster,base_cluster,entropy_ans_correct_mistral,entropy_ans_output_mistral,field_entropy_value_mistral,entropy_ans_output
0,ori_mmlu-jurisprudence,C,['There is no distinction between the two form...,law,Which of the following criticisms of Llewellyn...,NaN,1286,2,81,Legal Interpretation,Legal Theory Interpretations,True,3,0.390101,NaN
1,ori_mmlu-international_law,E,"['Article 19', 'Article 11', 'Article 12', 'Ar...",law,Which of the following articles are not qualif...,NaN,1293,4,38,Legal Interpretation,Constitutional Law,False,4,0.251793,NaN
2,ori_mmlu-management,D,"['Work delegation', 'Workload balancing', 'Wor...",business,As what is ensuring that one individual does n...,NaN,83,3,49,Economics & Finance MCQs,Business & Marketing Queries,False,2,0.304977,NaN
3,stemez-Business,J,"['$308.25', '$142.75', '$199.99', '$225.85', '...",business,Margaret Denault recently rented a truck to dr...,NaN,94,9,118,Economics & Finance MCQs,Business Finance Questions,False,9,0.341230,NaN
4,stemez-Business,I,"['$60,000', '$43,200', '$1,794', '$25,000', '$...",business,The tax rate in the town of Centerville is 11(...,NaN,104,8,102,Economics & Finance MCQs,Business Finance Questions,False,7,0.265020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,stemez-ElectronicCommunications,E,"['L{t} = (2 / s^3)', 'L{t} = (1 / s^3)', 'L{t}...",engineering,"Find the Laplace transform of f(t) = t, for t ...",NaN,11431,4,131,Engineering Calculations,Electric Circuit Calculations,True,5,0.158495,NaN
6012,stemez-ElectronicCommunications,G,"['10 kHz and 7 kHz', '22 kHz and 17 kHz', '20 ...",engineering,(a) Determine the bandwidth occupied by a freq...,NaN,11432,6,155,Engineering Calculations,Electric Circuit Calculations,False,4,0.303783,NaN
6013,stemez-Thermodynamics,A,"['Heat Transfer: - 421.74 kJ, Irreversibility:...",engineering,"A 0.1 m diameter, 0.1 m high solid copper cyli...",NaN,11437,0,282,Engineering Calculations,Heat Transfer Problems,False,,0.000000,INVALID
6014,stemez-Thermodynamics,I,"['(a) - 34.2 Btu, (b) - 0.0166 Btu/°R', '(a) -...",engineering,Two pounds of oxygen undergo a polytropic proc...,NaN,11440,8,359,Engineering Calculations,Heat Transfer Problems,False,4,0.335504,NaN
